In [7]:
import sys
import torch
from torchvision import transforms
sys.path.append("../src")
from torch.utils.data import DataLoader
from mucac_utils import MUCAC
from data_utils import UnlearningDataLoader
from seed import set_seed
set_seed()

In [24]:
def compute_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean**2) ** 0.5

    return mean, std

In [3]:
train = MUCAC(
    split="train",
    transform=transforms.Compose(
        [
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
        ]
    ),
)
train_loader = DataLoader(train, batch_size=1, shuffle=False, num_workers=4)
val = MUCAC(split="val", transform=transforms.ToTensor())
val_loader = DataLoader(val, batch_size=1, shuffle=False, num_workers=4)
test = MUCAC(split="test", transform=transforms.ToTensor())
test_loader = DataLoader(test, batch_size=1, shuffle=False, num_workers=4)

In [26]:
# train_mean, train_std = compute_mean_std(train_loader)
# val_mean, val_std = compute_mean_std(val_loader)
# test_mean, test_std = compute_mean_std(test_loader)

In [27]:
# print(f"Train mean: {train_mean}, std: {train_std}")
# print(f"Val mean: {val_mean}, std: {val_std}")
# print(f"Test mean: {test_mean}, std: {test_std}")

Train mean: tensor([0.4719, 0.3761, 0.3224]), std: tensor([0.3211, 0.2816, 0.2673])


In [8]:
UDL = UnlearningDataLoader("mucac", batch_size=128, image_size=128, seed=3407)